In [1]:
import time
import pandas as pd

from bs4 import BeautifulSoup
import urllib.request as req

import requests
import os
import datetime
import re

In [2]:
import sys
sys.path.append("../")

In [3]:
from index.season.season_index import get_seasons

In [4]:
# ディビジョンを選択してください:{v1_m, v2_m, v3_m, v1_w, v2_w}
division = 'v1_m'
# '年度を選択してください:{2019-20, 2018-19, 2017-18...}'
season = '2018-19'
# シーズンを選択してください:{regular, 2019-20(game1, game2, semi_final, final), 2014-15~2018-19(final6, final3, final),
# ~2013-14(semi_final, final)}
s_round = 'regular'
season += '_' + s_round
season

&#39;2018-19_regular&#39;

In [5]:
seasons = get_seasons(division)
print(seasons)

{&#39;283&#39;: &#39;2019-20_regular&#39;, &#39;296&#39;: &#39;2019-20_game1&#39;, &#39;297&#39;: &#39;2019-20_game2&#39;, &#39;298&#39;: &#39;2019-20_semi_final&#39;, &#39;275&#39;: &#39;2019-20_final&#39;, &#39;258&#39;: &#39;2018-19_regular&#39;, &#39;261&#39;: &#39;2018-19_final6&#39;, &#39;260&#39;: &#39;2018-19_final3&#39;, &#39;259&#39;: &#39;2018-19_final&#39;, &#39;241&#39;: &#39;2017-18_regular&#39;, &#39;239&#39;: &#39;2017-18_final6&#39;, &#39;240&#39;: &#39;2017-18_final3&#39;, &#39;242&#39;: &#39;2017-18_final&#39;, &#39;224&#39;: &#39;2016-17_regular&#39;, &#39;223&#39;: &#39;2016-17_final6&#39;, &#39;226&#39;: &#39;2016-17_final3&#39;, &#39;225&#39;: &#39;2016-17_final&#39;, &#39;206&#39;: &#39;2015-16_regular&#39;, &#39;209&#39;: &#39;2015-16_final6&#39;, &#39;208&#39;: &#39;2015-16_final3&#39;, &#39;207&#39;: &#39;2015-16_final&#39;, &#39;181&#39;: &#39;2014-15_regular&#39;, &#39;184&#39;: &#39;2014-15_final6&#39;, &#39;183&#39;: &#39;2014-15_final3&#39;, &#39;182&#39

In [6]:
os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}

seasons = get_seasons(division)

if not os.path.isdir(division):
            os.makedirs(division)

os.chdir(division)
print(os.getcwd())

for s_id, s_round in seasons.items():
    print(s_id, s_round)

    sets = ['1','2','3','4','5']
    game_all = None

    # ページ数を取得
    url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
    print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')
    hrefs = parse_html.find_all('a', href=re.compile("pg"))

    # for i in hrefs:
    #     print(i.text)
    if len(hrefs) > 0:
        pages = int(hrefs[-2].text)
        print(pages)

        all_list = []
        for pg in range(1,pages+1):
            url = 'https://www.vleague.jp/round/list/{0}?pg={1}'.format(s_id,pg)
            print(url)
            request = req.Request(url, headers=headers)
            response = req.urlopen(request)
            parse_html = BeautifulSoup(response, 'html.parser')
            tables = parse_html.find_all('table')

            url_list = []
            trs = tables[0].find_all('a', href=re.compile("/form/b"))
            for tr in trs:
                game_url = 'https://www.vleague.jp/'+ tr.attrs['href']
                url_list.append(game_url)
            print(url_list)

            for url in url_list:
                # print(url)
                html = requests.get(url, headers=headers)
                check = html.text
                if '試合データがありません' not in check:
                    try:
                        data = pd.read_html(check, header=[2])

                        for j in range(2):
                            stats = pd.DataFrame(data[j+4])
                            stats = stats.rename(columns=str)
                            col = stats.columns.values
                            data_item = []
                            for i in col:
                                l = i
                                data_item.append(l)
                            for i in range(1,len(data_item)-2):
                                data_item[-i] = data_item[-i-3]
                                # print(data_item)
                            for i in range(3):
                            #     print(data_item[i])
                                if data_item[i] == '出場数':
                                    data_item[i] = '背番号'
                                elif data_item[i] == '1':
                                    data_item[i] = 'リベロ'
                                else:
                                    data_item[i] = '名前'
                            #     print(i, data_item[i])

                            new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
                            new_stats = new_stats.rename(columns={
                                '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                                'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                                '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                                'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                                '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                                '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                                })
                            
                            request = req.Request(url, headers=headers)
                            response = req.urlopen(request)
                            parse_html = BeautifulSoup(response, 'html.parser')
                            table_ha = parse_html.find_all('table')[0]
                            td_ha = table_ha.find_all('td', class_='team')
                            # print(td_ha)
                            team = td_ha[j].text
                            # print('team={}'.format(team))
                            new_stats['アタック決定率'] = new_stats['アタック得点'] / new_stats['アタック打数']
                            new_stats.insert(13, 'アタック効果率',
                                (new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数'])
                            span = parse_html.find_all('span')
                            date = span[1].text.replace('/','-')
                            new_stats.insert(0, '試合日', date)
                            new_stats.insert(1, 'チーム', team)
                            new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
                            for by_set in sets:
                                new_stats[by_set] = new_stats[by_set].astype(str)
                            # 必要に応じてteamディレクトリを作成
                            team_dir = '{0}/{1}'.format(team,s_round)
                            if not os.path.isdir(team_dir):
                                os.makedirs(team_dir)
                            # new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,s_round,date), index=False, encoding='cp932')
                            # print(new_stats)
                            all_list.append(new_stats)
                    except:
                        pass
        game_all = pd.concat(all_list, ignore_index=True)
    else:
        all_list = []
        url = 'https://www.vleague.jp/round/list/{}'.format(s_id)
        print(url)
        request = req.Request(url, headers=headers)
        response = req.urlopen(request)
        parse_html = BeautifulSoup(response, 'html.parser')
        tables = parse_html.find_all('table')

        url_list = []
        trs = tables[0].find_all('a', href=re.compile("/form/b"))
        for tr in trs:
            game_url = 'https://www.vleague.jp/'+ tr.attrs['href']
            url_list.append(game_url)
        print(url_list)

        for url in url_list:
            # print(url)
            html = requests.get(url, headers=headers)
            check = html.text
            if '試合データがありません' not in check:
                try:
                    data = pd.read_html(check, header=[2])

                    for j in range(2):
                        stats = pd.DataFrame(data[j+4])
                        stats = stats.rename(columns=str)
                        col = stats.columns.values
                        data_item = []
                        for i in col:
                            l = i
                            data_item.append(l)
                        for i in range(1,len(data_item)-2):
                            data_item[-i] = data_item[-i-3]
                            # print(data_item)
                        for i in range(3):
                        #     print(data_item[i])
                            if data_item[i] == '出場数':
                                data_item[i] = '背番号'
                            elif data_item[i] == '1':
                                data_item[i] = 'リベロ'
                            else:
                                data_item[i] = '名前'
                        #     print(i, data_item[i])

                        new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
                        new_stats = new_stats.rename(columns={
                            '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                            'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                            '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                            'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                            '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                            '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                            })
                        
                        request = req.Request(url, headers=headers)
                        response = req.urlopen(request)
                        parse_html = BeautifulSoup(response, 'html.parser')
                        table_ha = parse_html.find_all('table')[0]
                        td_ha = table_ha.find_all('td', class_='team')
                        # print(td_ha)
                        team = td_ha[j].text
                        # print('team={}'.format(team))
                        new_stats['アタック決定率'] = (new_stats['アタック得点'] / new_stats['アタック打数']).round(1)
                        new_stats.insert(13, 'アタック効果率',
                            (new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数']).round(1)
                        span = parse_html.find_all('span')
                        date = span[1].text.replace('/','-')
                        new_stats.insert(0, '試合日', date)
                        new_stats.insert(1, 'チーム', team)
                        new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
                        for by_set in sets:
                            new_stats[by_set] = new_stats[by_set].astype(str)
                        # 必要に応じてteamディレクトリを作成
                        # team_dir = '{0}/{1}'.format(team,s_round)
                        # if not os.path.isdir(team_dir):
                        #     os.makedirs(team_dir)
                        # new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,s_round,date), index=False, encoding='cp932')
                        all_list.append(new_stats)
                except:
                    pass
        try:
            game_all = pd.concat(all_list, ignore_index=True)
            print(game_all)

        except:
            pass
    all_dir = 'all/{}'.format(s_round)
    if not os.path.isdir(all_dir):
        os.makedirs(all_dir)
    
    print(os.getcwd())
    try:
        daily_all = game_all[game_all['名前'] != 'チーム合計']
        print(daily_all)

        data = daily_all
        data.insert(1, '月', data['試合日'].str[5:7])
        data.set_index('試合日')
        monthly_all = data.groupby(['名前','月','チーム','背番号']).sum().reset_index()
        monthly_all['アタック決定率'] = ((monthly_all['アタック得点'] / monthly_all['アタック打数']) * 100).round(1)
        monthly_all['アタック効果率'] = (((monthly_all['アタック得点']-monthly_all['アタック失点']) / monthly_all['アタック打数']) * 100).round(1)
        monthly_all['バックアタック決定率'] = ((monthly_all['バックアタック得点'] / monthly_all['バックアタック打数']) * 100).round(1)
        monthly_all['アタックセット平均'] = (monthly_all['アタック得点'] / monthly_all['出場数']).round(1)
        monthly_all['ブロックセット平均'] = (monthly_all['ブロック得点'] / monthly_all['出場数']).round(1)
        monthly_all['サーブ効果率'] = ((monthly_all['サーブ得点'] * 100 + monthly_all['サーブ効果'] * 25 - monthly_all['サーブ失点'] * 25) / monthly_all['サーブ打数']).round(1)
        monthly_all['サーブレシーブ成功率'] = ((monthly_all['サーブレシーブ成功・優'] * 100 + monthly_all['サーブレシーブ成功・良'] * 50) / monthly_all['受数']).round(1)
        print(monthly_all)

        yearly_all = data.groupby(['名前','チーム','背番号']).sum().reset_index()
        yearly_all['アタック決定率'] = ((yearly_all['アタック得点'] / yearly_all['アタック打数']) * 100).round(1)
        yearly_all['アタック効果率'] = (((yearly_all['アタック得点']-yearly_all['アタック失点']) / yearly_all['アタック打数']) * 100).round(1)
        yearly_all['バックアタック決定率'] = ((yearly_all['バックアタック得点'] / yearly_all['バックアタック打数']) * 100).round(1)
        yearly_all['アタックセット平均'] = (yearly_all['アタック得点'] / yearly_all['出場数']).round(1)
        yearly_all['ブロックセット平均'] = (yearly_all['ブロック得点'] / yearly_all['出場数']).round(1)
        yearly_all['サーブ効果率'] = ((yearly_all['サーブ得点'] * 100 + yearly_all['サーブ効果'] * 25 - yearly_all['サーブ失点'] * 25) / yearly_all['サーブ打数']).round(1)
        print(yearly_all)

        game_all.to_csv('all/{}/game_all.csv'.format(s_round), index=False, encoding='cp932')
        daily_all.to_csv('all/{}/daily_all.csv'.format(s_round), index=False, encoding='cp932')
        monthly_all.to_csv('all/{}/monthly_all.csv'.format(s_round), index=False, encoding='cp932')
        yearly_all.to_csv('all/{}/yearly_all.csv'.format(s_round), index=False, encoding='cp932')
    except:
        pass

#39;https://www.vleague.jp//form/b/20175&#39;, &#39;https://www.vleague.jp//form/b/20173&#39;]
c:\Users\sasno\Desktop\MyPandas\vleague\v1_m
             試合日      チーム 背番号  リベロ    名前  出場数    1    2    3    4  ...  \
0     2000-12-15   東レアローズ   1  NaN  中野順平    0  nan  nan  nan  nan  ...   
1     2000-12-15   東レアローズ   3  NaN  加藤陽一    5    ■    ■    ■    ■  ...   
2     2000-12-15   東レアローズ   5  NaN  笠原紀久    1    ■  nan  nan  nan  ...   
3     2000-12-15   東レアローズ   6  NaN   山口誠    5    ■    ■    ■    ■  ...   
4     2000-12-15   東レアローズ   7  NaN  齋藤信治    5    □    ■    ■    ■  ...   
...          ...      ...  ..  ...   ...  ...  ...  ...  ...  ...  ...   
2333  2001-02-25  JTサンダーズ  11  NaN  山田忠一    3    ■    □    □  nan  ...   
2334  2001-02-25  JTサンダーズ  12    L  山本健之    3    ●    ●    ●  nan  ...   
2335  2001-02-25  JTサンダーズ  14  NaN  伊勢隆一    3    ■    ■    ■  nan  ...   
2336  2001-02-25  JTサンダーズ  15  NaN  小森誠治    1  nan    □  nan  nan  ...   
2337  2001-02-25  JTサンダーズ  16  NaN  下村英士    1 

In [ ]:
daily_all = game_all[game_all['名前']!='チーム合計']
data = daily_all
data.insert(1, '月', data['試合日'].str[5:7])
data.set_index('試合日')
monthly_all = data.groupby(['名前','月','チーム','背番号']).sum().reset_index()
monthly_all['アタック決定率'] = ((monthly_all['アタック得点'] / monthly_all['アタック打数']) * 100).round(1)
monthly_all['アタック効果率'] = (((monthly_all['アタック得点']-monthly_all['アタック失点']) / monthly_all['アタック打数']) * 100).round(1)
monthly_all['バックアタック決定率'] = ((monthly_all['バックアタック得点'] / monthly_all['バックアタック打数']) * 100).round(1)
monthly_all['アタックセット平均'] = (monthly_all['アタック得点'] / monthly_all['出場数']).round(1)
monthly_all['ブロックセット平均'] = (monthly_all['ブロック得点'] / monthly_all['出場数']).round(1)
monthly_all['サーブ効果率'] = ((monthly_all['サーブ得点'] * 100 + monthly_all['サーブ効果'] * 25 - monthly_all['サーブ失点'] * 25) / monthly_all['サーブ打数']).round(1)
monthly_all['サーブレシーブ成功率'] = ((monthly_all['サーブレシーブ成功・優'] * 100 + monthly_all['サーブレシーブ成功・良'] * 50) / monthly_all['受数']).round(1)
monthly_all

In [9]:
daily_all = game_all[game_all['名前']!='チーム合計']
data = daily_all
data.insert(1, '月', data['試合日'].str[5:7])
data.set_index('試合日')
yearly_all = data.groupby(['名前','チーム','背番号']).sum().reset_index()
yearly_all['アタック決定率'] = ((yearly_all['アタック得点'] / yearly_all['アタック打数']) * 100).round(1)
yearly_all['アタック効果率'] = (((yearly_all['アタック得点']-yearly_all['アタック失点']) / yearly_all['アタック打数']) * 100).round(1)
yearly_all['バックアタック決定率'] = ((yearly_all['バックアタック得点'] / yearly_all['バックアタック打数']) * 100).round(1)
yearly_all['アタックセット平均'] = (yearly_all['アタック得点'] / yearly_all['出場数']).round(1)
yearly_all['ブロックセット平均'] = (yearly_all['ブロック得点'] / yearly_all['出場数']).round(1)
yearly_all['サーブ効果率'] = ((yearly_all['サーブ得点'] * 100 + yearly_all['サーブ効果'] * 25 - yearly_all['サーブ失点'] * 25) / yearly_all['サーブ打数']).round(1)
yearly_all['サーブレシーブ成功率'] = ((yearly_all['サーブレシーブ成功・優'] * 100 + yearly_all['サーブレシーブ成功・良'] * 50) / yearly_all['受数']).round(1)
yearly_all

,名前,チーム,背番号,出場数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,バックアタック打数,...,サーブ失点,サーブ効果,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,バックアタック決定率,アタックセット平均,ブロックセット平均,サーブ効果率,サーブレシーブ成功率
0,上出哲也,近畿クラブスフィーダ,17,9,20,11,0,55.0,55.0,0,...,3,6,1,1,0,NaN,1.2,0.8,2.9,100.0
1,上田日登,千葉ZELVA,18,10,4,2,0,50.0,50.0,0,...,4,2,0,0,0,NaN,0.2,0.1,-2.9,NaN
2,下町浩二,奈良ドリーマーズ,4,43,157,65,12,41.4,33.8,0,...,8,28,4,2,0,NaN,1.5,0.6,7.8,50.0
3,中倉翼,兵庫デルフィーノ,21,0,0,0,0,NaN,NaN,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,中村郁弥,近畿クラブスフィーダ,26,2,1,0,0,0.0,0.0,0,...,0,0,0,0,0,NaN,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,長濱裕,きんでんトリニティーブリッツ,3,58,0,0,0,NaN,NaN,0,...,0,0,302,190,0,NaN,0.0,0.0,NaN,62.9
124,長田翼,VC長野トライデンツ,24,5,0,0,0,NaN,NaN,0,...,0,1,0,0,0,NaN,0.0,0.0,3.6,NaN
125,関根翔太,千葉ZELVA,28,31,50,16,4,32.0,24.0,6,...,17,16,0,0,0,16.7,0.5,0.1,6.6,NaN
126,鳥越裕喜,近畿クラブスフィーダ,14,24,187,89,18,47.6,38.0,50,...,13,23,2,2,0,44.0,3.7,0.4,6.4,100.0


In [ ]:
all_dir = 'all/{}'.format(season)
if not os.path.isdir(all_dir):
        os.makedirs(all_dir)

In [ ]:
game_all = pd.concat(all_list, ignore_index=True)
# game_all
game_all.to_csv('all/{}/game_all.csv'.format(season), index=False, encoding='cp932')

In [ ]:
os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}

seasons = get_seasons(season,division)

for s_id, s_round in seasons.items():
    if season == s_round:
        season_id = s_id
print(season_id, season)
if not os.path.isdir(division):
            os.makedirs(division)

os.chdir(division)
print(os.getcwd())

sets = ['1','2','3','4','5']

# ページ数を取得
url = 'https://www.vleague.jp/round/list/{}'.format(season_id)
print(url)
request = req.Request(url, headers=headers)
response = req.urlopen(request)
parse_html = BeautifulSoup(response, 'html.parser')
hrefs = parse_html.find_all('a', href=re.compile("pg"))

# for i in hrefs:
#     print(i.text)
pages = int(hrefs[-2].text)
print(pages)

all_list = []
for pg in range(1,pages+1):
    url = 'https://www.vleague.jp/round/list/{0}?pg={1}'.format(season_id,pg)
    print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')
    tables = parse_html.find_all('table')

    url_list = []
    tr = tables[0].find_all('a')
    for i in range(1,len(tr),2):
        game_url = 'https://www.vleague.jp/'+ tr[i].attrs['href']
        url_list.append(game_url)

    for url in url_list:
        # print(url)
        html = requests.get(url, headers=headers)
        data = pd.read_html(html.text, header=[2])

        for j in range(2):
            stats = pd.DataFrame(data[j+4])
            stats = stats.rename(columns=str)
            col = stats.columns.values
            data_item = []
            for i in col:
                l = i
                data_item.append(l)
            for i in range(1,len(data_item)-2):
                data_item[-i] = data_item[-i-3]
                # print(data_item)
            for i in range(3):
            #     print(data_item[i])
                if data_item[i] == '出場数':
                    data_item[i] = '背番号'
                elif data_item[i] == '1':
                    data_item[i] = 'リベロ'
                else:
                    data_item[i] = '名前'
            #     print(i, data_item[i])

            new_stats = stats.rename(columns={col[i]:data_item[i] for i in range(len(data_item))})
            new_stats = new_stats.rename(columns={
                '打数':'アタック打数', '得点':'アタック得点', '失点':'アタック失点', '決定率':'アタック決定率',
                'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', '得点.2':'ブロック得点',
                'セ平ット均.1':'ブロックセット平均', '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'
                })
            
            request = req.Request(url, headers=headers)
            response = req.urlopen(request)
            parse_html = BeautifulSoup(response, 'html.parser')
            table_ha = parse_html.find_all('table')[0]
            td_ha = table_ha.find_all('td', class_='team')
            # print(td_ha)
            team = td_ha[j].text
            # print('team={}'.format(team))
            new_stats['アタック決定率'] = new_stats['アタック得点'] / new_stats['アタック打数']
            new_stats.insert(13, 'アタック効果率',
                (new_stats['アタック得点'] - new_stats['アタック失点']) / new_stats['アタック打数'])
            span = parse_html.find_all('span')
            date = span[1].text.replace('/','-')
            new_stats.insert(0, '試合日', date)
            new_stats.insert(1, 'チーム', team)
            new_stats['名前'] = new_stats['名前'].str.replace('\u3000','')
            for by_set in sets:
                new_stats[by_set] = new_stats[by_set].astype(str)
            team_dir = '{0}/{1}'.format(team,season)
            if not os.path.isdir(team_dir):
                os.makedirs(team_dir)
            # new_stats.to_csv('{0}/{1}/{2}.csv'.format(team,season,date), index=False, encoding='cp932')
            all_list.append(new_stats)

In [ ]:
# ページ数を取得
url = 'https://www.vleague.jp/round/list/{}'.format(division_id)
print(url)
request = req.Request(url, headers=headers)
response = req.urlopen(request)
parse_html = BeautifulSoup(response, 'html.parser')
href = parse_html.find_all('a', href=re.compile("pg"))